In [16]:
import pandas as pd
import numpy as np 
import plotly as px
import os

In [17]:
current_directory = os.getcwd()
customer_data = pd.ExcelFile(f'{current_directory}\\Customer_Churn_Data_Large.xlsx')
customer_data

In [18]:
customer_demographics  = pd.read_excel(customer_data, 'Customer_Demographics')
customer_demographics

,CustomerID,Age,Gender,MaritalStatus,IncomeLevel
0,1,62,M,Single,Low
1,2,65,M,Married,Low
2,3,18,M,Single,Low
3,4,21,M,Widowed,Low
4,5,21,M,Divorced,Medium
...,...,...,...,...,...
995,996,54,F,Single,Low
996,997,19,M,Widowed,High
997,998,47,M,Married,Low
998,999,23,M,Widowed,High


In [19]:
transaction_history  = pd.read_excel(customer_data, 'Transaction_History')
transaction_history

,CustomerID,TransactionID,TransactionDate,AmountSpent,ProductCategory
0,1,7194,2022-03-27,416.50,Electronics
1,2,7250,2022-08-08,54.96,Clothing
2,2,9660,2022-07-25,197.50,Electronics
3,2,2998,2022-01-25,101.31,Furniture
4,2,1228,2022-07-24,397.37,Clothing
...,...,...,...,...,...
5049,1000,2724,2022-09-08,232.06,Groceries
5050,1000,2917,2022-12-13,324.98,Books
5051,1000,2979,2022-06-15,375.34,Groceries
5052,1000,8594,2022-04-08,166.73,Books


In [20]:
customer_service = pd.read_excel(customer_data, 'Customer_Service')
customer_service

,CustomerID,InteractionID,InteractionDate,InteractionType,ResolutionStatus
0,1,6363,2022-03-31,Inquiry,Resolved
1,2,3329,2022-03-17,Inquiry,Resolved
2,3,9976,2022-08-24,Inquiry,Resolved
3,4,7354,2022-11-18,Inquiry,Resolved
4,4,5393,2022-07-03,Inquiry,Unresolved
...,...,...,...,...,...
997,990,3671,2022-10-25,Complaint,Unresolved
998,992,2114,2022-09-29,Feedback,Unresolved
999,994,3087,2022-07-02,Complaint,Unresolved
1000,994,8508,2022-05-14,Complaint,Unresolved


In [37]:
online_activity = pd.read_excel(customer_data, 'Online_Activity')
online_activity.describe()

,CustomerID,LastLoginDate,LoginFrequency
count,1000.000000,1000,1000.000000
mean,500.500000,2023-07-05 21:28:48,25.912000
min,1.000000,2023-01-01 00:00:00,1.000000
25%,250.750000,2023-04-08 00:00:00,13.750000
50%,500.500000,2023-07-10 12:00:00,27.000000
75%,750.250000,2023-10-01 06:00:00,38.000000
max,1000.000000,2023-12-31 00:00:00,49.000000
std,288.819436,NaN,14.055953


In [ ]:
online_activity['ServiceUsage'].value_counts()

In [34]:
churn_status = pd.read_excel(customer_data, 'Churn_Status')
churn_status['ChurnStatus'] = churn_status['ChurnStatus'].astype('bool')
churn_status['ChurnStatus'] = churn_status['ChurnStatus'].replace({True: 'Churned', False: 'Active'})
churn_status['ChurnStatus'].value_counts()

ChurnStatus
Active     796
Churned    204
Name: count, dtype: int64